# Equivalent Circuit Parameter Identification

This notebook provides example usage for identifying stationary parameters for a two RC branch Thevenin model. The Thevenin model represents an electrochemical battery through an empirical circuit model capable of capturing the electrical response of the battery. This model can be extended with a thermal submodel, as well as additional parallel resistor-capacitor branches.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [1]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [2]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [3]:
np.random.seed(8)

## Importing Parameters

This can be completed by importing a JSON representation, such as the one in the PyBOP [examples](https://github.com/pybop-team/PyBOP/blob/develop/examples/parameters/initial_ecm_parameters.json). To import via JSON, either download the example file, or create your own and update the path below to reference the corresponding file.

In [4]:
# parameter_set = pybop.ParameterSet(
# json_path="examples/parameters/initial_ecm_parameters.json"
# )

Alternatively, define the initial parameter set with a dictionary. Ensure you have definitions for all R's, C's, and initial overpotentials for any additional RC elements.

In this example, we use the default parameter value for the "Open-circuit voltage [V] as provided by the original PyBaMM class. To update this, provide a function definition that matches this [function](https://github.com/pybamm-team/PyBaMM/blob/1943aa5ab2895b5378220595923dbae3d66b13c9/pybamm/input/parameters/ecm/example_set.py#L17).

In [5]:
parameter_set = pybamm.equivalent_circuit.Thevenin().default_parameter_values
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Current function [A]": 5,
        "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybamm.equivalent_circuit.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

## Identifying the Parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. First, we define the model class with two RC elements.

In [6]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

In this example, we are going to try to fit all five parameters at once. This isn't recommend for real-life application as identifiablity is challenging to guarantee with this large a parameter space. To do this, we define the `parameters` as,

In [7]:
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(2 - 4, 1e-4),
        bounds=[1e-4, 1e-2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(1e-4, 1e-4),
        bounds=[1e-5, 1e-2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(1e-4, 1e-4),
        bounds=[1e-5, 1e-2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(1e4, 5e2),
        bounds=[2.5e3, 5e4],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(1e4, 5e2),
        bounds=[2.5e3, 5e4],
    ),
]

Let's create some synthetic data to identify the parameters. This data is then corrupted with a small amount of Gaussian noise to represent some additional uncertainty in the measured values. We can then form the `pybop.Dataset` from this data.

In [8]:
sigma = 0.001
t_eval = np.arange(0, 900, 3)
sim = pybamm.Simulation(model, parameter_values=parameter_set)
sol = sim.solve(t_eval=[t_eval[0], t_eval[-1]], t_interp=t_eval)
corrupt_values = sol["Voltage [V]"].data + np.random.normal(0, sigma, len(t_eval))

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"].data,
        "Voltage [V]": corrupt_values,
    }
)
parameter_set.set_initial_state(f"{corrupt_values[0]} V")

/home/dibyendu/Oxford_Work/PyBOP/newenv/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'C1 [F]': 10000.0,
 'C2 [F]': 40000,
 'Cell capacity [A.h]': 5,
 'Cell thermal mass [J/K]': 1000,
 'Cell-jig heat transfer coefficient [W/K]': 10,
 'Current function [A]': 5,
 'Electron charge [C]': 1.602176634e-19,
 'Element-1 initial overpotential [V]': 0,
 'Element-2 initial overpotential [V]': 0,
 'Entropic change [V/K]': <function dUdT at 0x7bc0ae2eade0>,
 'Faraday constant [C.mol-1]': 96485.33212331001,
 'Ideal gas constant [J.K-1.mol-1]': 8.31446261815324,
 'Initial SoC': 0.49557321878199895,
 'Initial temperature [K]': 298.15,
 'Jig thermal mass [J/K]': 500,
 'Jig-air heat transfer coefficient [W/K]': 10,
 'Lower voltage cut-off [V]': 3.0,
 'Nominal cell capacity [A.h]': 5,
 'Open-circuit voltage [V]': <function ocv at 0x7bc0ae143e20>,
 'R0 [Ohm]': 0.001,
 'R1 [Ohm]': 0.0002,
 'R2 [Ohm]': 0.0003,
 'RCR lookup limit [A]': 340,
 'Upper voltage cut-off [V]': 4.2}

Now we construct the problem class. We add dataset, model and parameter set. Sum squared error is chosen as the cost for the optimization problem.

In [9]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_set)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the XNES method as it provides global optimisation capability.

In [10]:
options = pybop.PintsOptions(
    sigma=[1e-4, 1e-4, 1e-4, 10, 10],
    max_unchanged_iterations=50,
    max_iterations=250,
)
optim = pybop.XNES(problem, options=options)
results = optim.run()

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualize the results of the optimisation.

In [11]:
#pybop.plot.problem(problem, problem_inputs=results.x, title="Optimised Comparison");

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [12]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Conclusion

This notebook illustrates how to perform parameter estimation using CMA-ES in PyBOP, providing insights into the optimisation process through various visualisations.